In [113]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import save_image
import os

In [114]:
# Path to data
data_path = "./dataset/training-GAN/"

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Parameters
image_size = 64
batch_size = 64
noise_dim = 60
num_epochs = 150
learning_rate = 0.0001

# Transformations
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),  # Force all images to have the same size
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize to [-1, 1]
])

# Dataset
dataset = datasets.ImageFolder(root=data_path, transform=transform)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
num_classes = len(dataset.classes)

print(f"Classes: {dataset.classes}")
print(f"Number of classes: {len(dataset.classes)}")

Classes: ['China', 'Germany', 'India', 'Japan', 'UK', 'USA']
Number of classes: 6


In [115]:
# Generator
class Generator(nn.Module):
    def __init__(self, noise_dim, num_classes, image_channels):
        super(Generator, self).__init__()
        self.label_embedding = nn.Embedding(num_classes, num_classes)
        self.model = nn.Sequential(
            nn.Linear(noise_dim + num_classes, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, image_channels * image_size * image_size),
            nn.Tanh()
        )

    def forward(self, noise, labels):
        label_embed = self.label_embedding(labels)
        input = torch.cat((noise, label_embed), dim=1)
        output = self.model(input)
        return output.view(output.size(0), 3, image_size, image_size)

# Discriminator
class Discriminator(nn.Module):
    def __init__(self, num_classes, image_channels):
        super(Discriminator, self).__init__()
        self.label_embedding = nn.Embedding(num_classes, num_classes)
        self.model = nn.Sequential(
            nn.Linear(image_channels * image_size * image_size + num_classes, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, images, labels):
        images_flat = images.view(images.size(0), -1)
        label_embed = self.label_embedding(labels)
        input = torch.cat((images_flat, label_embed), dim=1)
        output = self.model(input)
        return output

# Initialize models
generator = Generator(noise_dim, num_classes, 3).to(device)
discriminator = Discriminator(num_classes, 3).to(device)

# Loss and optimizers
criterion = nn.BCELoss()
optimizer_G = optim.Adam(generator.parameters(), lr=learning_rate, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=learning_rate, betas=(0.5, 0.999))

# Training loop
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(data_loader):
        batch_size = images.size(0)
        images, labels = images.to(device), labels.to(device)
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)

        # Train Discriminator
        optimizer_D.zero_grad()
        outputs = discriminator(images, labels)
        d_loss_real = criterion(outputs, real_labels)
        noise = torch.randn(batch_size, noise_dim).to(device)
        fake_images = generator(noise, labels)
        outputs = discriminator(fake_images.detach(), labels)
        d_loss_fake = criterion(outputs, fake_labels)
        d_loss = d_loss_real + d_loss_fake
        d_loss.backward()
        optimizer_D.step()

        # Train Generator
        optimizer_G.zero_grad()
        outputs = discriminator(fake_images, labels)
        g_loss = criterion(outputs, real_labels)
        g_loss.backward()
        optimizer_G.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], D Loss: {d_loss.item()}, G Loss: {g_loss.item()}")

    # Save generated images every 10 epochs
    if (epoch + 1) % 10 == 0:
        with torch.no_grad():
            noise = torch.randn(16, noise_dim).to(device)
            labels = torch.randint(0, num_classes, (16,)).to(device)
            fake_images = generator(noise, labels)
            save_image(fake_images, f"generated_images_epoch_{epoch+1}.png", normalize=True)

Epoch [1/150], D Loss: 0.35599443316459656, G Loss: 1.694960594177246
Epoch [2/150], D Loss: 0.6302751302719116, G Loss: 1.0635716915130615
Epoch [3/150], D Loss: 0.6324901580810547, G Loss: 1.0196499824523926
Epoch [4/150], D Loss: 0.6364952325820923, G Loss: 1.1025069952011108
Epoch [5/150], D Loss: 0.6381392478942871, G Loss: 1.4775078296661377
Epoch [6/150], D Loss: 0.5270842909812927, G Loss: 1.4302788972854614
Epoch [7/150], D Loss: 0.5627069473266602, G Loss: 1.4136946201324463
Epoch [8/150], D Loss: 0.4247141480445862, G Loss: 1.4889781475067139
Epoch [9/150], D Loss: 0.43274739384651184, G Loss: 1.4742587804794312
Epoch [10/150], D Loss: 0.3912230134010315, G Loss: 1.565316915512085
Epoch [11/150], D Loss: 0.399790495634079, G Loss: 1.7438255548477173
Epoch [12/150], D Loss: 0.36344772577285767, G Loss: 2.002863883972168
Epoch [13/150], D Loss: 0.252550333738327, G Loss: 2.394333839416504
Epoch [14/150], D Loss: 0.2634652554988861, G Loss: 2.4849693775177
Epoch [15/150], D Los

In [173]:
# Function to generate and save a specific image
def generate_specific_image(generator, label, noise_dim, image_path="specific_image.png"):
    generator.eval()  # Set the generator to evaluation mode
    
    # Create noise vector and label tensor
    noise = torch.randn(1, noise_dim).to(device)  # Generate a single noise vector
    label_tensor = torch.tensor([label], dtype=torch.long).to(device)  # Convert label to tensor
    
    with torch.no_grad():
        # Generate image
        generated_image = generator(noise, label_tensor)
    
    # Save image
    save_image(generated_image, image_path, normalize=True)
    print(f"Generated image saved at: {image_path}")

# Example usage
print(dataset.classes)
specific_label = 1 # Replace with the desired label (e.g., class index 3)
generate_specific_image(generator, specific_label, noise_dim, image_path="generated_label_53.png")

['USA']


TypeError: Generator.forward() takes 2 positional arguments but 3 were given

In [175]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision.utils import save_image
import os

# Hyperparameters
batch_size = 64
image_size = 64   # Image size (64x64)
latent_dim = 100  # Latent vector size (random noise)
epochs = 100
learning_rate = 0.0002
beta1 = 0.5

# Generator Model (Fully Connected)
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.ReLU(True),
            nn.Linear(256, 512),
            nn.ReLU(True),
            nn.Linear(512, 1024),
            nn.ReLU(True),
            nn.Linear(1024, 3 * image_size * image_size),  # Output size should match flattened image
            nn.Tanh()  # To match the pixel values in the range [-1, 1]
        )

    def forward(self, z):
        x = self.model(z)
        x = x.view(-1, 3, image_size, image_size)  # Reshape to image shape (3, 64, 64)
        return x

# Discriminator Model (Fully Connected)
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(3 * image_size * image_size, 1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid()  # Output probability of real/fake
        )

    def forward(self, x):
        return self.model(x)

# DataLoader (Image loading from folder)
transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

# Replace `your_dataset_folder` with your folder path
dataset = datasets.ImageFolder(root="./dataset/training-GAN/US/", transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Initialize models, loss function, and optimizers
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

generator = Generator().to(device)
discriminator = Discriminator().to(device)

criterion = nn.BCELoss()

optimizer_g = optim.Adam(generator.parameters(), lr=learning_rate, betas=(beta1, 0.999))
optimizer_d = optim.Adam(discriminator.parameters(), lr=learning_rate, betas=(beta1, 0.999))

# Training Loop
for epoch in range(epochs):
    for i, (imgs, _) in enumerate(dataloader):
        real_imgs = imgs.to(device)
        batch_size = real_imgs.size(0)

        # Create labels
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)

        # Train the Discriminator
        optimizer_d.zero_grad()

        # Real images
        outputs = discriminator(real_imgs)
        d_loss_real = criterion(outputs, real_labels)
        d_loss_real.backward()

        # Fake images
        z = torch.randn(batch_size, latent_dim).to(device)
        fake_imgs = generator(z)
        outputs = discriminator(fake_imgs.detach())  # Detach to avoid training the generator
        d_loss_fake = criterion(outputs, fake_labels)
        d_loss_fake.backward()

        optimizer_d.step()

        # Train the Generator
        optimizer_g.zero_grad()

        outputs = discriminator(fake_imgs)
        g_loss = criterion(outputs, real_labels)  # Want the generator to fool the discriminator
        g_loss.backward()

        optimizer_g.step()

        # Print losses and save generated images
        if i % 100 == 0:
            print(f"Epoch [{epoch}/{epochs}], Step [{i}/{len(dataloader)}], "
                  f"D Loss: {d_loss_real.item() + d_loss_fake.item():.4f}, "
                  f"G Loss: {g_loss.item():.4f}")

    # Save generated images at the end of each epoch
    if epoch % 10 == 0:
        save_image(fake_imgs.data[:25], f"output_epoch_{epoch}.png", nrow=5, normalize=True)

# Optionally, save models
torch.save(generator.state_dict(), "generator.pth")
torch.save(discriminator.state_dict(), "discriminator.pth")


Epoch [0/100], Step [0/3], D Loss: 1.3950, G Loss: 0.6811
Epoch [1/100], Step [0/3], D Loss: 0.9470, G Loss: 0.6395
Epoch [2/100], Step [0/3], D Loss: 0.9179, G Loss: 0.6858
Epoch [3/100], Step [0/3], D Loss: 0.7930, G Loss: 0.8247
Epoch [4/100], Step [0/3], D Loss: 0.8370, G Loss: 0.8477
Epoch [5/100], Step [0/3], D Loss: 0.7703, G Loss: 0.7687
Epoch [6/100], Step [0/3], D Loss: 0.6981, G Loss: 0.7686
Epoch [7/100], Step [0/3], D Loss: 0.6818, G Loss: 0.7742
Epoch [8/100], Step [0/3], D Loss: 0.6448, G Loss: 0.8171
Epoch [9/100], Step [0/3], D Loss: 0.6113, G Loss: 0.8789
Epoch [10/100], Step [0/3], D Loss: 0.6296, G Loss: 0.8841
Epoch [11/100], Step [0/3], D Loss: 0.6398, G Loss: 0.8614
Epoch [12/100], Step [0/3], D Loss: 0.6465, G Loss: 0.8603
Epoch [13/100], Step [0/3], D Loss: 0.6004, G Loss: 0.8952
Epoch [14/100], Step [0/3], D Loss: 0.5978, G Loss: 0.9719
Epoch [15/100], Step [0/3], D Loss: 0.5592, G Loss: 0.9944
Epoch [16/100], Step [0/3], D Loss: 0.5665, G Loss: 0.9038
Epoch [